# 	面试题19	正则表达式匹配  

思路：
- 非确定有限状态机（NFA）
    - 当第二个字符不是‘*’的情况
        - 模式串和字符串匹配或者模式串为‘.’，继续匹配剩余字符，否则返回false
    - 当第二个字符是‘*’的情况
        - 如果‘*’前一个字符和字符串不匹配，则跳过该模式
        - 如果匹配，则有两种选择
            - 模式向后移动2位 （匹配0位）
            - 模式保持不变，字符串向后移动一位（匹配多位）
- 动态规划
    - $dp[i][j]$  表示的状态是 s 的前 i 项和 p 的前 j 项是否匹配
    - 当第二个字符不是‘*’的情况
        - 模式串和字符串匹配或者模式串为‘.’, $dp[i][j] = dp[i-1][j-1]$
    - 当第二个字符是‘*’的情况
        - 如果‘*’前一个字符和字符串不匹配，则跳过该模式, $dp[i][j] = dp[i][j-2]$
        - 如果匹配，则有两种选择
            - 当匹配0个时，如 ab和abb* ,这时我们需要去掉p中的b*后进行比较, $dp[i][j] = dp[i][j-2]$
            - 当匹配多个时，如abbb和ab*，我们需要将s[i]前面的与p重新比较, $dp[i][j] = dp[i-1][j]$
    - 以上情况都不匹配，则 $dp[i][j] = false$
    - $d p(i)(j)=\left\{\begin{array}{ll}d p(i-1)(j-1), & \mathrm{s}(\mathrm{i})=\mathrm{p}(\mathrm{j}) \text { or } \mathrm{p}(\mathrm{j})='.' \\ d p(i)(j-2), & \mathrm{p}(\mathrm{j})=^{*}, \mathrm{p}(\mathrm{j}-1) !=\mathrm{s}(\mathrm{i}) \\ d p(i-1)(j) \text { or } d p(i)(j-2), & \mathrm{p}(\mathrm{j})=*, \mathrm{p}(\mathrm{j}-1)=\mathrm{s}(\mathrm{i}) \text { or } \mathrm{p}(\mathrm{j}-1)='.' \\\text { False } & \text { else }\end{array}\right.$
    - 给s,p前面都加上一个字符‘#’，表示从空字符开始匹配
    - https://leetcode-cn.com/problems/zheng-ze-biao-da-shi-pi-pei-lcof/solution/hui-su-dong-tai-gui-hua-by-ml-zimingmeng/
    - https://leetcode-cn.com/problems/zheng-ze-biao-da-shi-pi-pei-lcof/solution/dong-tai-gui-hua-chao-xiang-xi-jie-da-you-fan-ru-j/
- 回溯法
    - 思路和上面的差不多，与非确定有限状态机都是使用递归

注意：
- 非确定有限状态机
    - 字符串s用完，模式串p还有剩余不代表匹配失败，e.g. string: '', pattern: 'c*'
- 动态规划
    - 字符串前面加符号，从空字符开始判断
    - if i == 0: 不仅表示字符串s为空情况，只考虑带‘*’的情况，也限制了它进入下面的判断，引起问题，如s=‘’，p=‘.’

仿照书上的思路改写的，先用的下标来定位，但无法解决 eg s：'a' p：'ab*',就想这种星号在最后的情况，和其他情况一起会造成下标不一致，从而字符串的超过范围

后来通过切片改进，感觉逻辑是对的，但提交时会超时

删除三个并集情况的第一个，能通过提交了，第一个情况存在重复了感觉，加多了递归造成超时

In [1]:
class Solution:
    def isMatch(self, s: str, p: str) -> bool:


        return self.matchCore(s, p)

    def matchCore(self, s, p):
        # 递归出口，都匹配完表示匹配成功
        if len(s) == 0 and len(p) == 0:
            return True
        # 字符串还剩余，模式串匹配完，则无法继续匹配，匹配失败
        if s.strip() and len(p) == 0:
            return False
        # 带*处理
        if len(p) > 1 and p[1] == '*':  
            if len(s) != 0:
                # 如果s和p第一个对应，返回三个情况的并集
                if s[0] == p[0] or p[0] == '.':
                    # s向后移动1个，模式向后移动2个
                    # 保持模式不变
                    # 模式向后移动2个
#                     return self.matchCore(s[1:], p[2:]) \
#                            or self.matchCore(s[1:], p) \
#                            or self.matchCore(s, p[2:]) 
                    return self.matchCore(s[1:], p) \
                           or self.matchCore(s, p[2:]) 
                # s和p不匹配，忽略这个模式
                else:
                    return self.matchCore(s, p[2:])
            # s用完，p还有，忽略这个模式
            else:
                return self.matchCore(s, p[2:])
        # 不带*处理
        elif len(s) != 0:
            # s和p对应或者p为字符‘.’
            if s[0] == p[0] or p[0] == '.':
                return self.matchCore(s[1:], p[1:])

        return False


s = Solution()

print(s.isMatch("", "a*"))


True


参考别人思路写的，没有超时有点奇怪，可能我那个判断多了递归太多

参考：https://leetcode-cn.com/problems/zheng-ze-biao-da-shi-pi-pei-lcof/solution/mian-shi-ti-19-zheng-ze-biao-da-shi-pi-pei-di-gui-/

In [2]:
class Solution:
    def isMatch(self, s: str, p: str) -> bool:

        if s is None or p is None:
            return False
        return self.matchCore(s, p)

    def matchCore(self, s, p):
        # s 和 p都为0 表示匹配完成
        if len(s) == 0 and len(p) == 0:
            return True
        # s还有字符，而模式串已经用完，匹配无法完成表示匹配失败
        if len(s) != 0 and len(p) == 0:
            return False
        # 针对带有*的处理
        if len(p) > 1 and p[1] == '*':
            # s和p第一个字符对应不上，直接跳过这个匹配
            # e.g. string: '', pattern: 'c*'
            # e.g. string: 'a', pattern: 'c*a'
            if self.matchCore(s, p[2:]):
                return True
            # s和p第一个字符对应
            # e.g. string: 'a', pattern: 'a*'
            if s and s[0] == p[0]:
                return self.matchCore(s[1:], p)
             # p第一个字符为‘.’
             # e.g. string: 'c', pattern: '.*'
            if s and p[0] == '.':
                return self.matchCore(s[1:], p)
        # 没有*的处理
        # 两种情况就是s和p的字符对应，或者p字符为‘.’都表示成功匹配
        # 字符串s和模式串p都进入下个字符的匹配
        elif s and (s[0] == p[0] or p[0] == '.'):
            return self.matchCore(s[1:], p[1:])
        #以上都没有，则表示匹配失败
        return False


s = Solution()
print(s.isMatch("", "a*"))

True


# 答案

## 非确定有限状态机

In [3]:
class Solution:
    def isMatch(self, s: str, p: str) -> bool:

        if s is None or p is None:
            return False
        return self.matchCore(s, p)

    def matchCore(self, s, p):

        if len(s) == 0 and len(p) == 0:
            return True
        if len(s) != 0 and len(p) == 0:
            return False

        if len(p) > 1 and p[1] == '*':
            if self.matchCore(s, p[2:]):
                return True
            if s and s[0] == p[0]:
                return self.matchCore(s[1:], p)
            if s and p[0] == '.':
                return self.matchCore(s[1:], p)
        elif s and (s[0] == p[0] or p[0] == '.'):
            return self.matchCore(s[1:], p[1:])
        return False

## 动态规划

In [4]:
class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        s, p = '#' + s, '#' + p
        s_len, p_len = len(s), len(p)
        dp = [[False] * p_len for _ in range(s_len)]
        dp[0][0] = True

        for i in range(s_len):
            for j in range(1, p_len):
                # 字符串s为空情况，只考虑带‘*’的情况
                if i == 0:
                    dp[i][j] = j > 1 and p[j] == '*' and dp[i][j - 2]
                # 模式串p和字符串s对应或模式串p为符合‘.’，就看前面的匹配结果
                elif p[j] == s[i] or p[j] == '.':
                    dp[i][j] = dp[i - 1][j - 1]
                # 模式串p为‘*’的情况
                elif p[j] == '*':
                    # 模式串‘*’前的字符和字符串s匹配，分为匹配0个和匹配多个，满足其中一个都行
                    if p[j - 1] == s[i] or p[j - 1] == '.':
                        dp[i][j] = dp[i - 1][j] or dp[i][j - 2]
                    # 模式串‘*’前的字符和字符串s不匹配，直接跳过该模式
                    else:
                        dp[i][j] = dp[i][j - 2]
                else:
                    dp[i][j] = False
        return dp[-1][-1]

## 回溯法

In [5]:
class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        if not p: return not s
        # 第一个字母是否匹配
        first_match = bool(s and p[0] in {s[0],'.'})
        # 如果 p 第二个字母是 *
        if len(p) >= 2 and p[1] == "*":
            return self.isMatch(s, p[2:]) or \
            first_match and self.isMatch(s[1:], p)
        else:
            return first_match and self.isMatch(s[1:], p[1:])